In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from models.shake_pyramidnet import ShakePyramidNet  # 모델 클래스 임포트
import time
from tqdm import tqdm  # 진행 바 표시용

# CUDA 메모리 관리 최적화 설정
torch.backends.cudnn.benchmark = False  # 벤치마크 끄기
torch.backends.cudnn.deterministic = True  # 결정적 알고리즘 사용
torch.cuda.empty_cache()  # CUDA 캐시 비우기

# 디바이스 설정 - CPU 옵션 추가
force_cpu = False  # True로 설정하면 강제로 CPU 사용
device = torch.device("cpu") if force_cpu else torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU 수: {torch.cuda.device_count()}")
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
    print(f"가용 CUDA 메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# 테스트 데이터셋 설정 (CIFAR-100)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

testset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(
    testset, 
    batch_size=128,  # 배치 크기 조정
    shuffle=False, 
    pin_memory=(device.type == 'cuda'),  # GPU 사용시에만 pin_memory 활성화
    num_workers=32
)

# 배치 수 계산하여 출력
total_batches = len(testloader)
print(f"총 테스트 배치 수: {total_batches}, 배치 크기: {testloader.batch_size}")
print(f"총 테스트 샘플 수: {len(testset)}")

# ShakePyramidNet 모델 생성 함수
def create_model():
    model = ShakePyramidNet(
        depth=110,
        alpha=270,
        label=100     # CIFAR-100이므로 100 클래스
    )
    return model

# 모델 평가 함수 - 진행 상태 표시 기능 추가
def evaluate_model(model_path, model_name):
    try:
        start_time = time.time()
        print(f"\n[{time.strftime('%H:%M:%S')}] {model_name} 평가 시작...")
        
        # 모델 생성
        model = create_model()
        
        # 저장된 가중치를 불러옴
        print(f"[{time.strftime('%H:%M:%S')}] 모델 가중치 로딩 중...")
        if device.type == 'cuda':
            state_dict = torch.load(model_path, map_location=device)
        else:
            state_dict = torch.load(model_path, map_location='cpu')
        
        # 'module.' 접두사 처리
        if all(k.startswith('module.') for k in state_dict.keys()):
            # GPU 사용 가능하고 여러 개일 경우 DataParallel 사용
            if device.type == 'cuda' and torch.cuda.device_count() > 1 and not force_cpu:
                model = nn.DataParallel(model)
            else:
                # CPU 모드이거나 GPU가 하나일 경우 접두사 제거
                state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
        
        # 가중치 로드
        model.load_state_dict(state_dict)
        print(f"[{time.strftime('%H:%M:%S')}] 모델 가중치 로딩 완료")
        
        # 디바이스로 모델 이동
        model = model.to(device)
        model.eval()
        
        correct_top1 = 0
        correct_top5 = 0
        total = 0
        
        cpu_fallback_count = 0  # CPU로 전환한 배치 수
        error_batch_count = 0   # 오류가 발생한 배치 수
        
        # 평가 과정에서 에러 핸들링 + 진행 상태 표시
        with torch.no_grad():
            # tqdm을 사용하여 진행 바 표시
            pbar = tqdm(testloader, desc=f"{model_name} 평가 중", unit="batch")
            for batch_idx, (inputs, labels) in enumerate(pbar):
                try:
                    # 5배치마다 진행 상황 출력
                    if batch_idx % 5 == 0:
                        print(f"[{time.strftime('%H:%M:%S')}] 배치 {batch_idx}/{total_batches} 처리 중...")
                    
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    
                    # Top-1 정확도
                    _, predicted = outputs.max(1)
                    batch_correct = predicted.eq(labels).sum().item()
                    total += labels.size(0)
                    correct_top1 += batch_correct
                    
                    # Top-5 정확도
                    _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
                    batch_correct_top5 = top5_idx.eq(labels.view(-1, 1).expand_as(top5_idx)).sum().item()
                    correct_top5 += batch_correct_top5
                    
                    # 현재 배치 정확도를 진행 바에 업데이트
                    batch_acc = 100. * batch_correct / labels.size(0)
                    pbar.set_postfix({"배치 정확도": f"{batch_acc:.2f}%", 
                                     "전체 정확도": f"{100. * correct_top1 / total:.2f}%"})
                    
                except RuntimeError as e:
                    print(f"\n[{time.strftime('%H:%M:%S')}] 배치 {batch_idx}/{total_batches} 처리 중 오류 발생: {e}")
                    error_batch_count += 1
                    
                    # CPU로 전환하여 재시도
                    if device.type == 'cuda':
                        print(f"[{time.strftime('%H:%M:%S')}] CPU로 전환하여 계속합니다...")
                        cpu_fallback_count += 1
                        inputs, labels = inputs.cpu(), labels.cpu()
                        model = model.cpu()
                        outputs = model(inputs)
                        
                        # Top-1 정확도
                        _, predicted = outputs.max(1)
                        batch_correct = predicted.eq(labels).sum().item()
                        total += labels.size(0)
                        correct_top1 += batch_correct
                        
                        # Top-5 정확도
                        _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
                        batch_correct_top5 = top5_idx.eq(labels.view(-1, 1).expand_as(top5_idx)).sum().item()
                        correct_top5 += batch_correct_top5
                        
                        # 현재 배치 정확도를 진행 바에 업데이트
                        batch_acc = 100. * batch_correct / labels.size(0)
                        pbar.set_postfix({"배치 정확도": f"{batch_acc:.2f}%", 
                                          "전체 정확도": f"{100. * correct_top1 / total:.2f}%",
                                          "CPU 전환": cpu_fallback_count})
                        
                        # 다시 GPU로 전환
                        model = model.to(device)
        
        accuracy_top1 = 100.0 * correct_top1 / total
        accuracy_top5 = 100.0 * correct_top5 / total
        
        evaluation_time = time.time() - start_time
        
        print(f'\n[{time.strftime("%H:%M:%S")}] {model_name} 평가 완료:')
        print(f'- Top-1 정확도: {accuracy_top1:.2f}%')
        print(f'- Top-5 정확도: {accuracy_top5:.2f}%')
        print(f'- 평가 소요 시간: {evaluation_time:.2f}초')
        print(f'- 오류 발생 배치 수: {error_batch_count}/{total_batches}')
        print(f'- CPU 전환 배치 수: {cpu_fallback_count}/{total_batches}')
        
        return accuracy_top1, accuracy_top5
    
    except Exception as e:
        print(f"\n[{time.strftime('%H:%M:%S')}] 모델 평가 중 예외 발생: {e}")
        return 0.0, 0.0

# 두 모델의 가중치 특성 비교
def analyze_model_weights(model_path, model_name):
    print(f"\n[{time.strftime('%H:%M:%S')}] {model_name} 가중치 분석 중...")
    
    # 파일 크기 확인
    import os
    file_size_mb = os.path.getsize(model_path) / (1024 * 1024)
    print(f"{model_name} 파일 크기: {file_size_mb:.2f} MB")
    
    # 가중치 통계 분석
    state_dict = torch.load(model_path, map_location='cpu')
    
    # 가중치 개수, 평균, 표준편차 계산
    total_params = 0
    total_weight_sum = 0
    total_abs_weight_sum = 0
    weight_std_sum = 0
    weight_count = 0
    
    for name, param in state_dict.items():
        if 'weight' in name:
            weight_count += 1
            param_count = param.numel()
            total_params += param_count
            
            # 가중치 통계
            param_mean = param.mean().item()
            param_abs_mean = param.abs().mean().item()
            param_std = param.std().item()
            
            total_weight_sum += param_mean * param_count
            total_abs_weight_sum += param_abs_mean * param_count
            weight_std_sum += param_std
    
    # 평균값 계산
    avg_weight = total_weight_sum / total_params
    avg_abs_weight = total_abs_weight_sum / total_params
    avg_std = weight_std_sum / weight_count
    
    print(f"{model_name} 총 파라미터 수: {total_params:,}")
    print(f"{model_name} 가중치 평균: {avg_weight:.6f}")
    print(f"{model_name} 가중치 절대값 평균: {avg_abs_weight:.6f}")
    print(f"{model_name} 가중치 표준편차 평균: {avg_std:.6f}")
    
    # 학습된 모델 여부 예측
    is_trained = avg_std > 0.01  # 일반적으로 학습된 모델은 표준편차가 더 큼
    print(f"{model_name}는 {'학습된 모델로 추정됩니다.' if is_trained else '학습되지 않은 모델로 추정됩니다.'}")
    
    return {
        'file_size_mb': file_size_mb,
        'total_params': total_params,
        'avg_weight': avg_weight,
        'avg_abs_weight': avg_abs_weight,
        'avg_std': avg_std,
        'is_trained': is_trained
    }

# 먼저 두 모델의 가중치를 분석하여 학습 여부 추정
print("\n------- 모델 가중치 분석 -------")
model1_path = 'best_model_shake_pyramidnet_1.pth'
model1_analysis = analyze_model_weights(model1_path, "모델 1")

model2_path = 'best_model_shake_pyramidnet_2.pth'
model2_analysis = analyze_model_weights(model2_path, "모델 2")

print("\n------- 가중치 분석 결과 비교 -------")
if model1_analysis['is_trained'] and not model2_analysis['is_trained']:
    print("모델 1이 학습된 모델이고, 모델 2는 학습되지 않은 모델로 추정됩니다.")
    better_model = model1_path
elif not model1_analysis['is_trained'] and model2_analysis['is_trained']:
    print("모델 2가 학습된 모델이고, 모델 1은 학습되지 않은 모델로 추정됩니다.")
    better_model = model2_path
elif model1_analysis['is_trained'] and model2_analysis['is_trained']:
    print("두 모델 모두 학습된 것으로 추정됩니다. 정확한 비교를 위해 정확도 평가를 진행합니다.")
    # 두 모델 평가 진행
    print("\n------- 모델 1 평가 -------")
    model1_acc_top1, model1_acc_top5 = evaluate_model(model1_path, "ShakePyramidNet 모델 1")

    # 메모리 정리
    torch.cuda.empty_cache()

    print("\n------- 모델 2 평가 -------")
    model2_acc_top1, model2_acc_top5 = evaluate_model(model2_path, "ShakePyramidNet 모델 2")

    # 결과 비교
    print("\n------- 결과 비교 -------")
    if model1_acc_top1 > model2_acc_top1:
        print(f"모델 1이 Top-1 정확도에서 {model1_acc_top1 - model2_acc_top1:.2f}% 더 우수합니다.")
        better_model = model1_path
    elif model2_acc_top1 > model1_acc_top1:
        print(f"모델 2가 Top-1 정확도에서 {model2_acc_top1 - model1_acc_top1:.2f}% 더 우수합니다.")
        better_model = model2_path
    else:
        print("두 모델의 Top-1 정확도가 동일합니다.")
        better_model = model1_path if model1_acc_top5 >= model2_acc_top5 else model2_path
else:
    print("두 모델 모두 학습되지 않은 것으로 추정됩니다.")
    better_model = "없음"

if better_model != "없음":
    print(f"\n더 나은 모델: {better_model}")
    print(f"이 모델을 앙상블이나 추론에 사용하세요.")
else:
    print("\n두 모델 모두 사용하기에 적합하지 않습니다. 모델을 다시 학습하세요.")

Using device: cuda
GPU 수: 2
GPU 이름: NVIDIA RTX A5000
가용 CUDA 메모리: 25.43 GB
Files already downloaded and verified
총 테스트 배치 수: 79, 배치 크기: 128
총 테스트 샘플 수: 10000

------- 모델 가중치 분석 -------

[11:18:44] 모델 1 가중치 분석 중...
모델 1 파일 크기: 109.30 MB
모델 1 총 파라미터 수: 28,486,308
모델 1 가중치 평균: 0.000047
모델 1 가중치 절대값 평균: 0.002872
모델 1 가중치 표준편차 평균: 0.034607
모델 1는 학습된 모델로 추정됩니다.

[11:18:44] 모델 2 가중치 분석 중...
모델 2 파일 크기: 109.31 MB
모델 2 총 파라미터 수: 28,486,308
모델 2 가중치 평균: 0.000872
모델 2 가중치 절대값 평균: 0.027161
모델 2 가중치 표준편차 평균: 0.018295
모델 2는 학습된 모델로 추정됩니다.

------- 가중치 분석 결과 비교 -------
두 모델 모두 학습된 것으로 추정됩니다. 정확한 비교를 위해 정확도 평가를 진행합니다.

------- 모델 1 평가 -------

[11:18:45] ShakePyramidNet 모델 1 평가 시작...
[11:18:46] 모델 가중치 로딩 중...
[11:18:46] 모델 가중치 로딩 완료


ShakePyramidNet 모델 1 평가 중:   0%|                                                                     | 0/79 [00:00<?, ?batch/s]

[11:18:47] 배치 0/79 처리 중...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a CuDNNError: cuDNN error: CUDNN_STATUS_BAD_PARAM
Exception raised from run_conv_plan at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:374 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x7f6c7474d897 in /usr/local/lib/python3.11/dist-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0xe1c861 (0x7f6c755ef861 in /usr/local/lib/python3.11/dist-packages/torch/lib/libtorch_cuda.so)
frame #2: <unknown function> + 0x1095d83 (0x7f6c75868d83 in /usr/local/lib/python3.11/dist-packages/torch/lib/libtorch_cuda.so)
frame #3: <unknown function> + 0x1097c2c (0x7f6c7586ac2c in /usr/local/lib/python3.11/dist-packages/torch/lib/libtorch_cuda.so)
frame #4: <unknown function> + 0x109817b (0x7f6c7586b17b in /usr/local/lib/python3.11/dist-packages/torch/lib/libtorch_cuda.so)
frame #5: <unknown function> + 0x107aca2 (0x7f6c7584dca2 in /usr/local/lib/pyth

[11:21:37] 배치 5/79 처리 중...


...

ShakePyramidNet 모델 2 평가 중:  25%|█████▌                | 20/79 [10:52<31:57, 32.50s/batch, 배치 정확도=7.81%, 전체 정확도=6.76%]

[12:12:13] 배치 20/79 처리 중...


ShakePyramidNet 모델 2 평가 중:  32%|██████▉               | 25/79 [13:33<29:06, 32.34s/batch, 배치 정확도=6.25%, 전체 정확도=6.78%]

[12:14:55] 배치 25/79 처리 중...


ShakePyramidNet 모델 2 평가 중:  37%|████████              | 29/79 [15:42<26:54, 32.28s/batch, 배치 정확도=5.47%, 전체 정확도=6.90%]